In [2]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

### PLAYER Stats

In [3]:
url = "https://www.pdga.com/player/149756/stats/2024"

req = requests.get(url)

In [7]:
tables = pd.read_html(req.text)

C:\Users\dlmaynard\AppData\Local\Temp\ipykernel_7292\290228984.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(req.text)


In [15]:
tables[3]

,Place,Points,Tournament,Tier,Dates
0,6,35.0,Nightmare at Water Works,C,24-Feb-2024
1,4,110.0,Rosedale Cup 2024 Presented By Dynamic Discs K...,C,10-Mar-2024
2,1,5.0,Nightmare in Paradise - Black Thorn,C,23-Mar-2024
3,4,25.0,Nightmare in Paradise - Woodhenge,C,23-Mar-2024
4,17,210.0,Midwest Amateur Championships,A,19-Jul to 21-Jul-2024


In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URL of the player's 2024 stats page
url = "https://www.pdga.com/player/149756/stats/2024"

# Send a GET request
response = requests.get(url)
response.raise_for_status()  # Raise error if request fails

# Parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")

# Find all tables on the page (tournament results are typically in <table> elements)
tables = soup.find_all("table")

# If there are multiple tables, you may need to inspect which one you want
# Here, we'll assume the first table contains the stats
table = tables[0]

# Extract headers
headers = [th.text.strip() for th in table.find_all("th")]

# Extract rows
rows = []
for tr in table.find_all("tr")[1:]:  # skip header row
    cells = tr.find_all("td")
    row = [cell.text.strip() for cell in cells]
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows, columns=headers)

# Optional: add Event ID if it's in the links
event_ids = []
for tr in table.find_all("tr")[1:]:
    link = tr.find("a")
    if link and "event" in link.get("href", ""):
        # Extract numeric ID from URL
        event_id = link["href"].split("/")[-1]
        event_ids.append(event_id)
    else:
        event_ids.append(None)

df["Event ID"] = event_ids

print(df)


               Division Tournaments  Points Prize Event ID
0  MPO · Mixed Pro Open           7  305.00   $93     None
1           Pro Totals:           7  305.00   $93     None


In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# PDGA player stats page
url = "https://www.pdga.com/player/149756/stats/2024"

# Request the page
response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

# Find all tables (MPO, MA1, etc.)
tables = soup.find_all("table")

all_dfs = []

for table in tables:
    # Extract headers
    headers = [th.text.strip() for th in table.find_all("th")]
    
    # Extract rows
    rows = []
    event_ids = []
    for tr in table.find_all("tr")[1:]:  # skip header row
        cells = tr.find_all("td")
        row = [cell.text.strip() for cell in cells]
        rows.append(row)
        
        # Try to find the event link in this row
        link = tr.find("a", href=True)
        if link and "/tour/event/" in link['href']:
            event_id = link['href'].split("/tour/event/")[1].split("#")[0]
            event_ids.append(event_id)
        else:
            event_ids.append(None)
    
    # Build DataFrame for this table
    df = pd.DataFrame(rows, columns=headers)
    df["Event ID"] = event_ids
    all_dfs.append(df)

# Combine all tables into a single DataFrame
final_df = pd.concat(all_dfs, ignore_index=True)

print(final_df)


                 Division Tournaments  Points Prize Event ID Place  \
0    MPO · Mixed Pro Open           7  305.00   $93     None   NaN   
1             Pro Totals:           7  305.00   $93     None   NaN   
2   MA1 · Mixed Amateur 1           5  385.00   NaN     None   NaN   
3              Am Totals:           5  385.00   NaN     None   NaN   
4                     NaN         NaN   30.00          81215     7   
5                     NaN         NaN   35.00          78305     4   
6                     NaN         NaN   45.00   $66    82373     1   
7                     NaN         NaN   55.00          79658    11   
8                     NaN         NaN   30.00   $27    78306     2   
9                     NaN         NaN   90.00          76913    69   
10                    NaN         NaN   20.00          78445     6   
11                    NaN         NaN   35.00   NaN    76347     6   
12                    NaN         NaN  110.00   NaN    78751     4   
13                  

In [26]:
final_df

,Division,Tournaments,Points,Prize,Event ID,Place,Tournament,Tier,Dates
0,MPO · Mixed Pro Open,7,305.00,$93,None,NaN,NaN,NaN,NaN
1,Pro Totals:,7,305.00,$93,None,NaN,NaN,NaN,NaN
2,MA1 · Mixed Amateur 1,5,385.00,NaN,None,NaN,NaN,NaN,NaN
3,Am Totals:,5,385.00,NaN,None,NaN,NaN,NaN,NaN
4,NaN,NaN,30.00,,81215,7,F3 @ Rosedale presented by UnderPar,C,19-Apr-2024
5,NaN,NaN,35.00,,78305,4,gsDiscGolf - Cinco De Mayo Flex,C,05-May-2024
6,NaN,NaN,45.00,$66,82373,1,The Hatchet Granny Flexer sponsored by Belton ...,C,31-May-2024
7,NaN,NaN,55.00,,79658,11,The Gyroscope presented by MVP Disc Sports,C,22-Jun-2024
8,NaN,NaN,30.00,$27,78306,2,gsDiscGolf - 4th of July Flex,C,04-Jul-2024
9,NaN,NaN,90.00,,76913,69,Crane Insurance Presents the St. Louis Open Sp...,A,20-Sep to 22-Sep-2024


### Event - Round API example


In [ ]:
headers = {"User-Agent": "Mozilla/5.0"}
### Event - Round API example
EVENT = 'https://www.pdga.com/apps/tournament/live-api/live_results_fetch_round?TournID=88152&Division=MPO&Round=1'

r = requests.get(EVENT, headers=headers)
data = r.json()
df = pd.DataFrame(data['data']['scores'])
df.loc[df['PDGANum'] == 149756]